Давайте поставим эксперимент и проверим, так ли хороша на самом деле эта нормализованная инициализация. Для этого зададим простую полносвязную модель и будем оценивать точность на тестовом множестве в датасете MNIST. Сначала импортируем все необходимое из Keras:

In [1]:
from keras.models import Sequential
from keras.layers import Dense

C:\Users\Art\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Как и в TensorFlow, в Keras набор данных MNIST доступен «из коробки»:

In [2]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 26s 2us/step


Однако на этот раз правильные ответы заданы в виде цифр, и нам придется самостоятельно перекодировать их в виде векторов. Для этого можно использовать модуль np_utils, входящий в состав Keras:

In [3]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

Теперь осталось для удобства перевести матрицы X_train и X_test из целочисленных значений на отрезке [0,255] к вещественным на [0,1] (нормализовать), а также сделать из квадратных изображений размера 28 х 28 пикселов одномерные векторы длины 784; это значит, что сами тензоры X_train и X_test будут иметь размерность (число примеров) х 784:

In [9]:
x_train = x_train.reshape([-1, 28*28]) / 255.
x_test = x_test.reshape([-1, 28*28]) / 255.

Все, данные готовы. Поскольку мы собираемся определять сразу две одинаковые модели, различающиеся только способом инициализации весов, давайте сразу объявим соответствующую функцию.

In [32]:
def create_model(init):
    model = Sequential()
    model.add(Dense(100, input_shape=(28*28,), activation='tanh', kernel_initializer=init))
    model.add(Dense(100, activation='tanh', kernel_initializer=init))
    model.add(Dense(100, activation='tanh', kernel_initializer=init))
    model.add(Dense(100, activation='tanh', kernel_initializer=init))
    model.add(Dense(10, activation='softmax', kernel_initializer=init))
    return model

В этом коде функция create_model принимает на вход текстовый параметр, который интерпретируется как тип инициализации. Для нашего эксперимента это будут значения uniform и glorot_normal. А возвращает функция простую полносвязную модель с четырьмя промежуточными слоями, каждый размера 100. Везде, кроме последнего слоя, мы использум в качестве функции активации гиперболический тангенс, а в последнем слое — softmax, так как собираемся использовать в качестве функции потерь перекрестную энтропию.  
Процесс компиляции модели и ее обучения задается очень просто:

In [33]:
uniform_model = create_model("uniform")
uniform_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
uniform_model.fit(x_train, Y_train, 
                  batch_size=64, epochs=30, verbose=1, validation_data=(x_test, Y_test))

glorot_model = create_model("glorot_normal")
glorot_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
glorot_model.fit(x_train, Y_train, 
                  batch_size=64, epochs=30, verbose=1, validation_data=(x_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/30
60000/60000 [==============================] - 12s 201us/step - loss: 2.3010 - acc: 0.1118 - val_loss: 2.2991 - val_acc: 0.1135
Epoch 2/30
60000/60000 [==============================] - 12s 208us/step - loss: 2.2970 - acc: 0.1124 - val_loss: 2.2935 - val_acc: 0.1135
Epoch 3/30
60000/60000 [==============================] - 11s 185us/step - loss: 2.2795 - acc: 0.1693 - val_loss: 2.2333 - val_acc: 0.2624
Epoch 4/30
60000/60000 [==============================] - 12s 197us/step - loss: 1.8645 - acc: 0.2908 - val_loss: 1.4092 - val_acc: 0.4201
Epoch 5/30
60000/60000 [==============================] - 12s 196us/step - loss: 1.2485 - acc: 0.5189 - val_loss: 1.1231 - val_acc: 0.6213
Epoch 6/30
60000/60000 [==============================] - 12s 197us/step - loss: 0.9634 - acc: 0.6840 - val_loss: 0.7922 - val_acc: 0.7473
Epoch 7/30
60000/60000 [==============================] - 12s 193us/step - loss: 0.7194 - acc: 0.7722 - val_loss: 0.

60000/60000 [==============================] - 11s 190us/step - loss: 0.0623 - acc: 0.9822 - val_loss: 0.0930 - val_acc: 0.9701
Epoch 29/30
60000/60000 [==============================] - 12s 194us/step - loss: 0.0599 - acc: 0.9829 - val_loss: 0.0903 - val_acc: 0.9713
Epoch 30/30
60000/60000 [==============================] - 12s 194us/step - loss: 0.0573 - acc: 0.9838 - val_loss: 0.0924 - val_acc: 0.9702
